# Final fit

## Generate histograms

In [1]:
import zdb
import glob
import os
import oyaml as yaml
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dftools
import scipy
import pysge
from tqdm.auto import tqdm

plt.style.use('cms')
plt.rcParams.update({
    "figure.dpi": 150,
    "figure.figsize": (4, 3),
    "lines.markersize": 3,
    "errorbar.capsize": 2,
    "font.size": 10.,
})

Welcome to JupyROOT 6.18/00


In [5]:
def generate_yaml(hists, selection, filepath, tables={"central": "Events"}, systs=[], mc=False):
    hists_def = {
        k: [
            {"table": "df: '{table_name}'"},
            {"varname0": "df: '{}'".format(k)},
            {"selection": "df: '{selection_name}'"},
            {"parent": "df: df.parent"},
            {"binvar0": "df: df.{}_bin".format(k)},
            {"count": "df: 1."},
            {"sum_w": "df: {weight}"},
            {"sum_ww": "df: df.sum_w**2"},
        ] for k in hists
    }
    
    if mc:
        for key, hdef in hists_def.items():
            for label, vari in systs:
                hdef.extend([
                    {"sum_w_{}".format(label): "df: df.sum_w*({})".format(vari)},
                    {"sum_ww_{}".format(label): "df: (df.sum_w*({}))**2".format(vari)},
                ])
    
    monojet_selection = selection + ["METTriggered", "MinDPhiJ1234METnoX>0.5"]
    monojetqcd_selection = selection + ["METTriggered", "MinDPhiJ1234METnoX<=0.5"]
    singlemuon_selection = selection + ["METTriggered", "MinDPhiJ1234METnoX>0.5", "nMuonSelection==1", "MTW>30.", "MTW<125."]
    singlemuonplus_selection = selection + ["METTriggered", "MinDPhiJ1234METnoX>0.5", "nMuonSelection==1", "MTW>30.", "MTW<125.", "LeptonCharge==1."]
    singlemuonminus_selection = selection + ["METTriggered", "MinDPhiJ1234METnoX>0.5", "nMuonSelection==1", "MTW>30.", "MTW<125.", "LeptonCharge==-1."]
    singlemuonqcd_selection = selection + ["METTriggered", "MinDPhiJ1234METnoX<=0.5", "nMuonSelection==1", "MTW>30.", "MTW<125."]
    doublemuon_selection = selection + ["METTriggered", "MinDPhiJ1234METnoX>0.5", "nMuonSelection==2", "MLL>71.", "MLL<111.", "LeptonCharge==0."]
    doublemuonqcd_selection = selection + ["METTriggered", "MinDPhiJ1234METnoX<=0.5", "nMuonSelection==2", "MLL>71.", "MLL<111.", "LeptonCharge==0."]
    singleelectron_selection = selection + ["SingleElectronTriggered", "MinDPhiJ1234METnoX>0.5", "nElectronSelection==1", "MTW>30.", "MTW<125.", "MET_pt>100."]
    singleelectronplus_selection = selection + ["SingleElectronTriggered", "MinDPhiJ1234METnoX>0.5", "nElectronSelection==1", "MTW>30.", "MTW<125.", "MET_pt>100.", "LeptonCharge==1."]
    singleelectronminus_selection = selection + ["SingleElectronTriggered", "MinDPhiJ1234METnoX>0.5", "nElectronSelection==1", "MTW>30.", "MTW<125.", "MET_pt>100.", "LeptonCharge==-1."]
    singleelectronqcd_selection = selection + ["SingleElectronTriggered", "MinDPhiJ1234METnoX<=0.5", "nElectronSelection==1", "MTW>30.", "MTW<125.", "MET_pt>100."]
    doubleelectron_selection = selection + ["SingleElectronTriggered", "MinDPhiJ1234METnoX>0.5", "nElectronSelection==2", "MLL>71.", "MLL<111.", "LeptonCharge==0."]
    doubleelectronqcd_selection = selection + ["SingleElectronTriggered", "MinDPhiJ1234METnoX<=0.5", "nElectronSelection==2", "MLL>71.", "MLL<111.", "LeptonCharge==0."]
    singletau_selection = selection + ["METTriggered", "MinDPhiJ1234METnoX>0.5", "nTauSelection==1"]
    singlemuonmu_selection = selection + ["SingleMuonTriggered", "MinDPhiJ1234METnoX>0.5", "nMuonSelection==1", "MTW>30.", "MTW<125."]
    singlemuonmuqcd_selection = selection + ["SingleMuonTriggered", "MinDPhiJ1234METnoX<=0.5", "nMuonSelection==1", "MTW>30.", "MTW<125."]
    doublemuonmu_selection = selection + ["SingleMuonTriggered", "MinDPhiJ1234METnoX>0.5", "nMuonSelection==2", "MLL>71.", "MLL<111.", "LeptonCharge==0."]
    doublemuonmuqcd_selection = selection + ["SingleMuonTriggered", "MinDPhiJ1234METnoX<=0.5", "nMuonSelection==2", "MLL>71.", "MLL<111.", "LeptonCharge==0."]
    if not mc:
        monojet_selection.extend(["nMuonVeto==0", "nElectronVeto==0", "nTauVeto==0"])
        monojetqcd_selection.extend(["nMuonVeto==0", "nElectronVeto==0", "nTauVeto==0"])
        singlemuon_selection.extend(["nMuonVeto==1", "nElectronVeto==0", "nTauVeto==0"])
        singlemuonplus_selection.extend(["nMuonVeto==1", "nElectronVeto==0", "nTauVeto==0"])
        singlemuonminus_selection.extend(["nMuonVeto==1", "nElectronVeto==0", "nTauVeto==0"])
        singlemuonqcd_selection.extend(["nMuonVeto==1", "nElectronVeto==0", "nTauVeto==0"])
        doublemuon_selection.extend(["nMuonVeto==2", "nElectronVeto==0", "nTauVeto==0"])
        doublemuonqcd_selection.extend(["nMuonVeto==2", "nElectronVeto==0", "nTauVeto==0"])
        singleelectron_selection.extend(["nMuonVeto==0", "nElectronVeto==1", "nTauVeto==0"])
        singleelectronplus_selection.extend(["nMuonVeto==0", "nElectronVeto==1", "nTauVeto==0"])
        singleelectronminus_selection.extend(["nMuonVeto==0", "nElectronVeto==1", "nTauVeto==0"])
        singleelectronqcd_selection.extend(["nMuonVeto==0", "nElectronVeto==1", "nTauVeto==0"])
        doubleelectron_selection.extend(["nMuonVeto==0", "nElectronVeto==2", "nTauVeto==0"])
        doubleelectronqcd_selection.extend(["nMuonVeto==0", "nElectronVeto==2", "nTauVeto==0"])
        singletau_selection.extend(["nMuonVeto==0", "nElectronVeto==0", "nTauVeto==1"])
        singlemuonmu_selection.extend(["nMuonVeto==1", "nElectronVeto==0", "nTauVeto==0"])
        singlemuonmuqcd_selection.extend(["nMuonVeto==1", "nElectronVeto==0", "nTauVeto==0"])
        doublemuonmu_selection.extend(["nMuonVeto==2", "nElectronVeto==0", "nTauVeto==0"])
        doublemuonmuqcd_selection.extend(["nMuonVeto==2", "nElectronVeto==0", "nTauVeto==0"])
    else:
        monojet_selection.extend(["nTauSelection==0"])
        monojetqcd_selection.extend(["nTauSelection==0"])
        singlemuon_selection.extend(["nTauSelection==0"])
        singlemuonplus_selection.extend(["nTauSelection==0"])
        singlemuonminus_selection.extend(["nTauSelection==0"])
        singlemuonqcd_selection.extend(["nTauSelection==0"])
        doublemuon_selection.extend(["nTauSelection==0"])
        doublemuonqcd_selection.extend(["nTauSelection==0"])
        singleelectron_selection.extend(["nTauSelection==0"])
        singleelectronplus_selection.extend(["nTauSelection==0"])
        singleelectronminus_selection.extend(["nTauSelection==0"])
        singleelectronqcd_selection.extend(["nTauSelection==0"])
        doubleelectron_selection.extend(["nTauSelection==0"])
        doubleelectronqcd_selection.extend(["nTauSelection==0"])
        singlemuonmu_selection.extend(["nTauSelection==0"])
        singlemuonmuqcd_selection.extend(["nTauSelection==0"])
        doublemuonmu_selection.extend(["nTauSelection==0"])
        doublemuonmuqcd_selection.extend(["nTauSelection==0"])
        

    cutflows = {
        "Monojet": {
            "selection_name": "Monojet",
            "selection": "(" + ") & (".join(monojet_selection) + ")",
            "weight": "1." if not mc else "df.Weight_Monojet",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                 "MTW", "MLL", "Muon", "Electron", "Tau",
            ])],
        },
        "MonojetQCD": {
            "selection_name": "MonojetQCD",
            "selection": "(" + ") & (".join(monojetqcd_selection) + ")",
            "weight": "1." if not mc else "df.Weight_MonojetQCD",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                 "MTW", "MLL", "Muon", "Electron", "Tau",
            ])],
        },
        "SingleMuon": {
            "selection_name": "SingleMuon",
            "selection": "(" + ") & (".join(singlemuon_selection) + ")",
            "weight": "1." if not mc else "df.Weight_SingleMuon",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MLL", "SecondMuon", "Electron", "Tau",
            ])],
        },
        "SingleMuonPlus": {
            "selection_name": "SingleMuonPlus",
            "selection": "(" + ") & (".join(singlemuonplus_selection) + ")",
            "weight": "1." if not mc else "df.Weight_SingleMuonPlus",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MLL", "SecondMuon", "Electron", "Tau",
            ])],
        },
        "SingleMuonMinus": {
            "selection_name": "SingleMuonMinus",
            "selection": "(" + ") & (".join(singlemuonminus_selection) + ")",
            "weight": "1." if not mc else "df.Weight_SingleMuonMinus",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MLL", "SecondMuon", "Electron", "Tau",
            ])],
        },
        "SingleMuonQCD": {
            "selection_name": "SingleMuonQCD",
            "selection": "(" + ") & (".join(singlemuonqcd_selection) + ")",
            "weight": "1." if not mc else "df.Weight_SingleMuonQCD",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MLL", "SecondMuon", "Electron", "Tau",
            ])],
        },
        "DoubleMuon": {
            "selection_name": "DoubleMuon",
            "selection": "(" + ") & (".join(doublemuon_selection) + ")",
            "weight": "1." if not mc else "df.Weight_DoubleMuon",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MTW", "Electron", "Tau",
            ])],
        },
        "SingleElectron": {
            "selection_name": "SingleElectron",
            "selection": "(" + ") & (".join(singleelectron_selection) + ")",
            "weight": "1." if not mc else "df.Weight_SingleElectron",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MLL", "SecondElectron", "Muon", "Tau",
            ])],
        },
        "SingleElectronPlus": {
            "selection_name": "SingleElectronPlus",
            "selection": "(" + ") & (".join(singleelectronplus_selection) + ")",
            "weight": "1." if not mc else "df.Weight_SingleElectronPlus",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MLL", "SecondElectron", "Muon", "Tau",
            ])],
        },
        "SingleElectronMinus": {
            "selection_name": "SingleElectronMinus",
            "selection": "(" + ") & (".join(singleelectronminus_selection) + ")",
            "weight": "1." if not mc else "df.Weight_SingleElectronMinus",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MLL", "SecondElectron", "Muon", "Tau",
            ])],
        },
        "SingleElectronQCD": {
            "selection_name": "SingleElectronQCD",
            "selection": "(" + ") & (".join(singleelectronqcd_selection) + ")",
            "weight": "1." if not mc else "df.Weight_SingleElectronQCD",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MLL", "SecondElectron", "Muon", "Tau",
            ])],
        },
        "DoubleElectron": {
            "selection_name": "DoubleElectron",
            "selection": "(" + ") & (".join(doubleelectron_selection) + ")",
            "weight": "1." if not mc else "df.Weight_DoubleElectron",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MTW", "Muon", "Tau",
            ])],
        },
        "SingleTau": {
            "selection_name": "SingleTau",
            "selection": "(" + ") & (".join(singletau_selection) + ")",
            "weight": "1." if not mc else "df.Weight_SingleTau",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MTW", "MLL", "Muon", "Electron",
            ])],
        },
        "SingleMuonMu": {
            "selection_name": "SingleMuonMu",
            "selection": "(" + ") & (".join(singlemuonmu_selection) + ")",
            "weight": "1." if not mc else "df.Weight_SingleMuonMu",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MLL", "SecondMuon", "Electron", "Tau",
            ])],
        },
        "SingleMuonMuQCD": {
            "selection_name": "SingleMuonMuQCD",
            "selection": "(" + ") & (".join(singlemuonmuqcd_selection) + ")",
            "weight": "1." if not mc else "df.Weight_SingleMuonMuQCD",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MLL", "SecondMuon", "Electron", "Tau",
            ])],
        },
        "DoubleMuonMu": {
            "selection_name": "DoubleMuonMu",
            "selection": "(" + ") & (".join(doublemuonmu_selection) + ")",
            "weight": "1." if not mc else "df.Weight_DoubleMuonMu",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MTW", "Electron", "Tau",
            ])],
        },
    }

    met_triggers = [
        "df.HLT_PFMETNoMu90_PFMHTNoMu90_IDTight",
        "df.HLT_PFMETNoMu100_PFMHTNoMu100_IDTight",
        "df.HLT_PFMETNoMu110_PFMHTNoMu110_IDTight",
        "df.HLT_PFMETNoMu120_PFMHTNoMu120_IDTight",
        "(df.HLT_PFMET170_NotCleaned & (df.run<276282))",
        "(df.HLT_PFMET170_BeamHaloCleaned & (df.run<276282))",
        "df.HLT_PFMET170_HBHECleaned",
        "df.HLT_PFMET170_HBHE_BeamHaloCleaned",
        "df.HLT_MET75_IsoTrk50",
    ]
    if mc:
        met_triggers = [
            "df.HLT_PFMETNoMu90_PFMHTNoMu90_IDTight",
            "df.HLT_PFMETNoMu100_PFMHTNoMu100_IDTight",
            "df.HLT_PFMETNoMu110_PFMHTNoMu110_IDTight",
            "df.HLT_PFMETNoMu120_PFMHTNoMu120_IDTight",
            "df.HLT_PFMET170_NotCleaned",
            "df.HLT_PFMET170_BeamHaloCleaned",
            "df.HLT_PFMET170_HBHECleaned",
            "df.HLT_PFMET170_HBHE_BeamHaloCleaned",
            "df.HLT_MET75_IsoTrk50",
        ]
        
    singlemuon_triggers = ["df.HLT_IsoMu24", "df.HLT_IsoTkMu24"]
    singleelectron_triggers = [
        "df.HLT_Ele27_WPTight_Gsf",
        "df.HLT_Ele115_CaloIdVT_GsfTrkIdT",
    ]

    cfg_eval_bins = [
        {"METnoX_pt_bin": "df: np.floor(df.METnoX_pt/5.)*5."},
    ]
    
    if mc:
        weight = [
            "df.WeightXsLumi",
            "df.WeightQcdEwk",
            "df.WeightPU",
            "df.WeightPreFiring",
            "df.WeightOneMinusPhotonVetoIdLoose",
            "df.WeightOneMinusPhotonVetoPixelSeedVeto",
            "df.WeightOneMinusJetBVetoIdMedium",
        ]
        weight_ele_veto = [
            "df.WeightOneMinusElectronVetoNoSelectionIdIsoVeto",
            "df.WeightOneMinusElectronSelectionIdIsoTight",
            "df.WeightOneMinusElectronVetoReco",
        ]
        weight_ele_sele = [
            "df.WeightElectronSelectionIdIsoTight",
            "df.WeightElectronSelectionReco",
            "df.WeightOneMinusElectronVetoNoSelectionIdIsoVeto",
            "df.WeightOneMinusElectronVetoNoSelectionReco",
        ]
        weight_mu_veto = [
            "df.WeightOneMinusMuonVetoNoSelectionIdLoose",
            "df.WeightOneMinusMuonVetoNoSelectionIsoLoose",
            "df.WeightOneMinusMuonSelectionIdTight",
            "df.WeightOneMinusMuonSelectionIsoTight",
        ]
        weight_mu_sele = [
            "df.WeightMuonSelectionIdTight",
            "df.WeightMuonSelectionIsoTight",
            "df.WeightOneMinusMuonVetoNoSelectionIdLoose",
            "df.WeightOneMinusMuonVetoNoSelectionIsoLoose",
        ]
        weight_tau_veto = [
            #"df.WeightOneMinusTauSelectionIdTight",
            "df.WeightOneMinusTauVetoNoSelectionIdVLoose",
        ]
        weight_tau_sele = [
            #"df.WeightTauSelectionIdTight",
            "df.WeightOneMinusTauVetoNoSelectionIdVLoose",
        ]
        weight_met_trig = ["df.WeightMETTrig",]
        weight_singleele_trig = ["df.WeightSingleElectronTrig"]
        weight_singlemu_trig = ["df.WeightSingleMuonTrig"]
        
        cfg_eval_bins.extend([
            {"parent": "df: np.where( df.parent.isin(['WJetsToLNu', 'DYJetsToLL']), np.where( df.parent=='WJetsToLNu', np.where( df.LeptonCharge>0, np.where( df.LeptonIsElectron, np.full_like(df.parent, 'WJetsToEPlusNu'), np.where( df.LeptonIsMuon, np.full_like(df.parent, 'WJetsToMuPlusNu'), np.where( df.LeptonIsTau, np.where( df.nGenTauL==0, np.full_like(df.parent, 'WJetsToTauHPlusNu'), np.where( df.nGenTauL==1, np.full_like(df.parent, 'WJetsToTauLPlusNu'), np.full_like(df.parent, 'WJetsToTauPlusNu'), ), ), np.full_like(df.parent, 'WJetsToLPlusNu'), ), ), ), np.where( df.LeptonIsElectron, np.full_like(df.parent, 'WJetsToEMinusNu'), np.where( df.LeptonIsMuon, np.full_like(df.parent, 'WJetsToMuMinusNu'), np.where( df.LeptonIsTau, np.where( df.nGenTauL==0, np.full_like(df.parent, 'WJetsToTauHMinusNu'), np.where( df.nGenTauL==1, np.full_like(df.parent, 'WJetsToTauLMinusNu'), np.full_like(df.parent, 'WJetsToTauMinusNu'), ), ), np.full_like(df.parent, 'WJetsToLMinusNu'), ), ), ), ), np.where( df.parent=='DYJetsToLL', np.where(  df.LeptonIsElectron, np.full_like(df.parent, 'DYJetsToEE'), np.where( df.LeptonIsMuon, np.full_like(df.parent, 'DYJetsToMuMu'), np.where( df.LeptonIsTau, np.where( df.nGenTauL==0, np.full_like(df.parent, 'DYJetsToTauHTauH'), np.where( df.nGenTauL==1, np.full_like(df.parent, 'DYJetsToTauHTauL'), np.where( df.nGenTauL==2, np.full_like(df.parent, 'DYJetsToTauLTauL'), np.full_like(df.parent, 'DYJetsToTauTau'), ), ), ), np.full_like(df.parent, 'DYJetsToLL'), ), ), ), df.parent, ), ), df.parent, )"},
            {"Weight_Monojet": "df: (" + ")*(".join(weight+weight_ele_veto+weight_mu_veto+weight_tau_veto+weight_met_trig)+")"},
            {"Weight_MonojetQCD": "df: df.Weight_Monojet"},
            {"Weight_SingleMuon": "df: (" + ")*(".join(weight+weight_ele_veto+weight_mu_sele+weight_tau_veto+weight_met_trig)+")"},
            {"Weight_SingleMuonPlus": "df: df.Weight_SingleMuon"},
            {"Weight_SingleMuonMinus": "df: df.Weight_SingleMuon"},
            {"Weight_SingleMuonQCD": "df: df.Weight_SingleMuon"},
            {"Weight_DoubleMuon": "df: df.Weight_SingleMuon"},
            {"Weight_SingleElectron": "df: (" + ")*(".join(weight+weight_ele_sele+weight_mu_veto+weight_tau_veto+weight_singleele_trig)+")"},
            {"Weight_SingleElectronPlus": "df: df.Weight_SingleElectron"},
            {"Weight_SingleElectronMinus": "df: df.Weight_SingleElectron"},
            {"Weight_SingleElectronQCD": "df: df.Weight_SingleElectron"},
            {"Weight_DoubleElectron": "df: df.Weight_SingleElectron"},
            {"Weight_SingleTau": "df: (" + ")*(".join(weight+weight_ele_veto+weight_mu_veto+weight_tau_sele+weight_met_trig)+")"},
            {"Weight_SingleMuonMu": "df: (" + ")*(".join(weight+weight_ele_veto+weight_mu_sele+weight_tau_veto+weight_singlemu_trig)+")"},
            {"Weight_SingleMuonMuQCD": "df: df.Weight_SingleMuonMu"},
            {"Weight_DoubleMuonMu": "df: df.Weight_SingleMuonMu"},
        ])
        
    cfg_eval_bins.extend([
        {"METTriggered": "df: (" + ") | (".join(met_triggers) + ")"},
        {"SingleMuonTriggered": "df: (" + ") | (".join(singlemuon_triggers) + ")"},
        {"SingleElectronTriggered": "df: (" + ") | (".join(singleelectron_triggers) + ")"},
    ])

    cfg = {
        "query": {
            "groupby": ["table", "varname0", "selection", "parent", "binvar0"],
            "tables": tables,
            "aliases": {},
            "eval": cfg_eval_bins,
            "cutflows": cutflows,
            "hists": hists_def,
        },
        "files": sorted(p for p in glob.glob(filepath)),
    }
    
    return cfg

In [6]:
hists = ["METnoX_pt"]
systs = [
    ("d1kqcdUp", "df.WeightQcdEwk_d1kqcdUp/df.WeightQcdEwk"),
    ("d1kqcdDown", "df.WeightQcdEwk_d1kqcdDown/df.WeightQcdEwk"),
    ("d2kqcdUp", "df.WeightQcdEwk_d2kqcdUp/df.WeightQcdEwk"),
    ("d2kqcdDown", "df.WeightQcdEwk_d2kqcdDown/df.WeightQcdEwk"),
    ("d3kqcdUp", "df.WeightQcdEwk_d3kqcdUp/df.WeightQcdEwk"),
    ("d3kqcdDown", "df.WeightQcdEwk_d3kqcdDown/df.WeightQcdEwk"),
    ("d1kewUp", "df.WeightQcdEwk_d1kewUp/df.WeightQcdEwk"),
    ("d1kewDown", "df.WeightQcdEwk_d1kewDown/df.WeightQcdEwk"),
    ("d2kewzUp", "df.WeightQcdEwk_d2kewzUp/df.WeightQcdEwk"),
    ("d2kewzDown", "df.WeightQcdEwk_d2kewzDown/df.WeightQcdEwk"),
    ("d2kewwUp", "df.WeightQcdEwk_d2kewwUp/df.WeightQcdEwk"),
    ("d2kewwDown", "df.WeightQcdEwk_d2kewwDown/df.WeightQcdEwk"),
    ("d3kewzUp", "df.WeightQcdEwk_d3kewzUp/df.WeightQcdEwk"),
    ("d3kewzDown", "df.WeightQcdEwk_d3kewzDown/df.WeightQcdEwk"),
    ("d3kewwUp", "df.WeightQcdEwk_d3kewwUp/df.WeightQcdEwk"),
    ("d3kewwDown", "df.WeightQcdEwk_d3kewwDown/df.WeightQcdEwk"),
    ("dkmixUp", "df.WeightQcdEwk_dkmixUp/df.WeightQcdEwk"),
    ("dkmixDown", "df.WeightQcdEwk_dkmixDown/df.WeightQcdEwk"),
    ("pileupUp", "df.WeightPU_pileupUp/df.WeightPU"),
    ("pileupDown", "df.WeightPU_pileupDown/df.WeightPU"), 
    ("prefiringUp", "df.WeightPreFiring_prefiringUp/df.WeightPreFiring"),
    ("prefiringDown", "df.WeightPreFiring_prefiringDown/df.WeightPreFiring"),
    ("muonTrigUp", "np.where(df.selection.str.contains('MuonMu'), df.WeightSingleMuonTrig_muonTrigUp/df.WeightSingleMuonTrig, np.ones_like(df.WeightSingleMuonTrig))"),
    ("muonTrigDown", "np.where(df.selection.str.contains('MuonMu'), df.WeightSingleMuonTrig_muonTrigDown/df.WeightSingleMuonTrig, np.ones_like(df.WeightSingleMuonTrig))"),
    ("muonIdLooseStatUp", 'df.WeightOneMinusMuonVetoNoSelectionIdLoose_muonIdLooseStatUp/df.WeightOneMinusMuonVetoNoSelectionIdLoose'),
    ("muonIdLooseStatDown", 'df.WeightOneMinusMuonVetoNoSelectionIdLoose_muonIdLooseStatDown/df.WeightOneMinusMuonVetoNoSelectionIdLoose'),
    ("muonIdLooseSystUp", 'df.WeightOneMinusMuonVetoNoSelectionIdLoose_muonIdLooseSystUp/df.WeightOneMinusMuonVetoNoSelectionIdLoose'),
    ("muonIdLooseSystDown", 'df.WeightOneMinusMuonVetoNoSelectionIdLoose_muonIdLooseSystDown/df.WeightOneMinusMuonVetoNoSelectionIdLoose'),
    ("muonIsoLooseStatUp", 'df.WeightOneMinusMuonVetoNoSelectionIsoLoose_muonIsoLooseStatUp/df.WeightOneMinusMuonVetoNoSelectionIsoLoose'),
    ("muonIsoLooseStatDown", 'df.WeightOneMinusMuonVetoNoSelectionIsoLoose_muonIsoLooseStatDown/df.WeightOneMinusMuonVetoNoSelectionIsoLoose'),
    ("muonIsoLooseSystUp", 'df.WeightOneMinusMuonVetoNoSelectionIsoLoose_muonIsoLooseSystUp/df.WeightOneMinusMuonVetoNoSelectionIsoLoose'),
    ("muonIsoLooseSystDown", 'df.WeightOneMinusMuonVetoNoSelectionIsoLoose_muonIsoLooseSystDown/df.WeightOneMinusMuonVetoNoSelectionIsoLoose'),
    ("muonIdTightStatUp", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIdTight_muonIdTightStatUp/df.WeightMuonSelectionIdTight, df.WeightOneMinusMuonSelectionIdTight_muonIdTightStatUp/df.WeightOneMinusMuonSelectionIdTight)"),
    ("muonIdTightStatDown", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIdTight_muonIdTightStatDown/df.WeightMuonSelectionIdTight, df.WeightOneMinusMuonSelectionIdTight_muonIdTightStatDown/df.WeightOneMinusMuonSelectionIdTight)"),
    ("muonIdTightSystUp", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIdTight_muonIdTightSystUp/df.WeightMuonSelectionIdTight, df.WeightOneMinusMuonSelectionIdTight_muonIdTightSystUp/df.WeightOneMinusMuonSelectionIdTight)"),
    ("muonIdTightSystDown", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIdTight_muonIdTightSystDown/df.WeightMuonSelectionIdTight, df.WeightOneMinusMuonSelectionIdTight_muonIdTightSystDown/df.WeightOneMinusMuonSelectionIdTight)"),
    ("muonIsoTightStatUp", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIsoTight_muonIsoTightStatUp/df.WeightMuonSelectionIsoTight, df.WeightOneMinusMuonSelectionIsoTight_muonIsoTightStatUp/df.WeightOneMinusMuonSelectionIsoTight)"),
    ("muonIsoTightStatDown", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIsoTight_muonIsoTightStatDown/df.WeightMuonSelectionIsoTight, df.WeightOneMinusMuonSelectionIsoTight_muonIsoTightStatDown/df.WeightOneMinusMuonSelectionIsoTight)"),
    ("muonIsoTightSystUp", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIsoTight_muonIsoTightSystUp/df.WeightMuonSelectionIsoTight, df.WeightOneMinusMuonSelectionIsoTight_muonIsoTightSystUp/df.WeightOneMinusMuonSelectionIsoTight)"),
    ("muonIsoTightSystDown", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIsoTight_muonIsoTightSystDown/df.WeightMuonSelectionIsoTight, df.WeightOneMinusMuonSelectionIsoTight_muonIsoTightSystDown/df.WeightOneMinusMuonSelectionIsoTight)"),
    ("eleTrigUp", "np.where(df.selection.str.contains('Electron'), df.WeightSingleElectronTrig_eleTrigUp/df.WeightSingleElectronTrig, np.ones_like(df.WeightSingleElectronTrig))"),
    ("eleTrigDown", "np.where(df.selection.str.contains('Electron'), df.WeightSingleElectronTrig_eleTrigDown/df.WeightSingleElectronTrig, np.ones_like(df.WeightSingleElectronTrig))"),
    ("eleIdIsoVetoUp", 'df.WeightOneMinusElectronVetoNoSelectionIdIsoVeto_eleIdIsoVetoUp/df.WeightOneMinusElectronVetoNoSelectionIdIsoVeto'),
    ("eleIdIsoVetoDown", 'df.WeightOneMinusElectronVetoNoSelectionIdIsoVeto_eleIdIsoVetoDown/df.WeightOneMinusElectronVetoNoSelectionIdIsoVeto'),
    ("eleIdIsoTightUp", "np.where(df.selection.str.contains('Electron'), df.WeightElectronSelectionIdIsoTight_eleIdIsoTightUp/df.WeightElectronSelectionIdIsoTight, df.WeightOneMinusElectronSelectionIdIsoTight_eleIdIsoTightUp/df.WeightOneMinusElectronSelectionIdIsoTight)"),
    ("eleIdIsoTightDown", "np.where(df.selection.str.contains('Electron'), df.WeightElectronSelectionIdIsoTight_eleIdIsoTightDown/df.WeightElectronSelectionIdIsoTight, df.WeightOneMinusElectronSelectionIdIsoTight_eleIdIsoTightDown/df.WeightOneMinusElectronSelectionIdIsoTight)"),
    ("eleRecoUp", "np.where(df.selection.str.contains('Electron'), df.WeightElectronSelectionReco_eleRecoUp*df.WeightOneMinusElectronVetoNoSelectionReco_eleRecoUp/(df.WeightElectronSelectionReco*df.WeightOneMinusElectronVetoNoSelectionReco), df.WeightOneMinusElectronVetoReco_eleRecoUp/df.WeightOneMinusElectronVetoReco)"),
    ("eleRecoDown", "np.where(df.selection.str.contains('Electron'), df.WeightElectronSelectionReco_eleRecoDown*df.WeightOneMinusElectronVetoNoSelectionReco_eleRecoDown/(df.WeightElectronSelectionReco*df.WeightOneMinusElectronVetoNoSelectionReco), df.WeightOneMinusElectronVetoReco_eleRecoDown/df.WeightOneMinusElectronVetoReco)"),
    ("photonIdLooseUp", "df.WeightOneMinusPhotonVetoIdLoose_photonIdLooseUp/df.WeightOneMinusPhotonVetoIdLoose"),
    ("photonIdLooseDown", "df.WeightOneMinusPhotonVetoIdLoose_photonIdLooseDown/df.WeightOneMinusPhotonVetoIdLoose"),
    ("photonPixelSeedVetoUp", "df.WeightOneMinusPhotonVetoPixelSeedVeto_photonPixelSeedVetoUp/df.WeightOneMinusPhotonVetoPixelSeedVeto"),
    ("photonPixelSeedVetoDown", "df.WeightOneMinusPhotonVetoPixelSeedVeto_photonPixelSeedVetoDown/df.WeightOneMinusPhotonVetoPixelSeedVeto"),
    ("tauIdVLooseUp", "df.WeightOneMinusTauVetoNoSelectionIdVLoose_tauIdVLooseUp/df.WeightOneMinusTauVetoNoSelectionIdVLoose"),
    ("tauIdVLooseDown", "df.WeightOneMinusTauVetoNoSelectionIdVLoose_tauIdVLooseDown/df.WeightOneMinusTauVetoNoSelectionIdVLoose"),
    #("tauIdTightUp", "np.where(df.selection.str.contains('Tau'), df.WeightTauSelectionIdTight_tauIdTightUp/df.WeightTauSelectionIdTight, df.WeightOneMinusTauSelectionIdTight_tauIdTightUp/df.WeightOneMinusTauSelectionIdTight)"),
    #("tauIdTightDown", "np.where(df.selection.str.contains('Tau'), df.WeightTauSelectionIdTight_tauIdTightDown/df.WeightTauSelectionIdTight, df.WeightOneMinusTauSelectionIdTight_tauIdTightDown/df.WeightOneMinusTauSelectionIdTight)"),
    ("tauIdTightUp", "np.where(df.selection.str.contains('Tau'), df.WeightTauSelectionIdTight_tauIdTightUp, df.WeightOneMinusTauSelectionIdTight_tauIdTightUp)"),
    ("tauIdTightDown", "np.where(df.selection.str.contains('Tau'), df.WeightTauSelectionIdTight_tauIdTightDown, df.WeightOneMinusTauSelectionIdTight_tauIdTightDown)"),
    ("btagSFUp", 'df.WeightOneMinusJetBVetoIdMedium_btagSFUp/df.WeightOneMinusJetBVetoIdMedium'),
    ("btagSFDown", 'df.WeightOneMinusJetBVetoIdMedium_btagSFDown/df.WeightOneMinusJetBVetoIdMedium'),
    ("metTrigMuonMultiplicitySystUp", "np.where(~(df.selection.str.contains('Electron') | df.selection.str.contains('MuonMu')), df.WeightMETTrig_muonMultiplicitySystUp/df.WeightMETTrig, np.ones_like(df.WeightMETTrig))"),
    ("metTrigMuonMultiplicitySystDown", "np.where(~(df.selection.str.contains('Electron') | df.selection.str.contains('MuonMu')), df.WeightMETTrig_muonMultiplicitySystDown/df.WeightMETTrig, np.ones_like(df.WeightMETTrig))"),
    ("metTrigReferenceTriggerSystUp", "np.where(~(df.selection.str.contains('Electron') | df.selection.str.contains('MuonMu')), df.WeightMETTrig_referenceTriggerSystUp/df.WeightMETTrig, np.ones_like(df.WeightMETTrig))"),
    ("metTrigReferenceTriggerSystDown", "np.where(~(df.selection.str.contains('Electron') | df.selection.str.contains('MuonMu')), df.WeightMETTrig_referenceTriggerSystDown/df.WeightMETTrig, np.ones_like(df.WeightMETTrig))"),
    ("metTrigRegionSystUp", "np.where(~(df.selection.str.contains('Electron') | df.selection.str.contains('MuonMu')), df.WeightMETTrig_regionSystUp/df.WeightMETTrig, np.ones_like(df.WeightMETTrig))"),
    ("metTrigRegionSystDown", "np.where(~(df.selection.str.contains('Electron') | df.selection.str.contains('MuonMu')), df.WeightMETTrig_regionSystDown/df.WeightMETTrig, np.ones_like(df.WeightMETTrig))"),
    ("alphasUp", "df.LHEPdfWeight101"),
    ("alphasDown", "df.LHEPdfWeight102"),
] + [
    ("lheScaleWeight{}".format(idx), "np.where(~(df.parent.str.contains('ZJetsTo') | df.parent.str.contains('WJetsTo') | df.parent.str.contains('DYJetsTo') | df.parent.str.contains('GStarJetsTo')), df.LHEScaleWeight{}, np.ones_like(df.LHEScaleWeight0))".format(idx))
    for idx in (0, 1, 3, 5, 7, 8)
] + [
    ("lhePdfWeight{}".format(idx), "df.LHEPdfWeight{}".format(idx))
    for idx in range(100)
]

data_selection = [
    "IsCertified", "Flag_goodVertices", "Flag_globalSuperTightHalo2016Filter",
    "Flag_HBHENoiseFilter", "Flag_HBHENoiseIsoFilter", "Flag_EcalDeadCellTriggerPrimitiveFilter",
    "Flag_BadPFMuonFilter", "Flag_eeBadScFilter", "MET_dCaloMET<0.5",
    "nJetSelection>0", "nJetSelection==nJetVeto", "LeadJetSelection_chHEF>0.1",
    "LeadJetSelection_neHEF<0.8", "LeadJetSelection_pt>200.", "nPhotonVeto==0",
    "nBJetVeto==0", "METnoX_pt>200."
]
mc_selection = [
    "(parent!='EWKV2Jets' | nGenBosonSelection==1)", "Flag_goodVertices", "Flag_globalSuperTightHalo2016Filter",
    "Flag_HBHENoiseFilter", "Flag_HBHENoiseIsoFilter", "Flag_EcalDeadCellTriggerPrimitiveFilter",
    "Flag_BadPFMuonFilter", "MET_dCaloMET<0.5", "nJetSelection>0",
    "nJetSelection==nJetVeto", "LeadJetSelection_chHEF>0.1", "LeadJetSelection_neHEF<0.8",
    "LeadJetSelection_pt>200.", "METnoX_pt>200.",
]

cfg_data = generate_yaml(
    hists, data_selection,
    "/vols/cms/sdb15/Analysis/ZinvWidth/databases/skims/2019/09_Sep/20_skims/data/*.h5",
    mc=False,
)
with open("configs/data.yaml", "w") as f:
    yaml.dump(cfg_data, f, indent=4)
    
cfg_mc = generate_yaml(
    hists, mc_selection,
    "/vols/cms/sdb15/Analysis/ZinvWidth/databases/skims/2019/09_Sep/14_skims/mc/*.h5",
    systs=systs,
    mc=True,
)
with open("configs/mc.yaml", "w") as f:
    yaml.dump(cfg_mc, f, indent=4)

cfg_mc_jes = generate_yaml(
    hists, mc_selection,
    "/vols/cms/sdb15/Analysis/ZinvWidth/databases/skims/2019/09_Sep/20_skims/mc_jes/*.h5",
    tables={"jesTotal10": "Events_jesTotal10", "jesTotal20": "Events_jesTotal20", "jesTotal30": "Events_jesTotal30", "jesTotal40": "Events_jesTotal40", "jesTotal50": "Events_jesTotal50", "jesTotal60": "Events_jesTotal60", "jesTotal70": "Events_jesTotal70", "jesTotal80": "Events_jesTotal80", "jesTotal90": "Events_jesTotal90"},
    mc=True,
)
with open("configs/mc_jes.yaml", "w") as f:
    yaml.dump(cfg_mc_jes, f, indent=4)
    
cfg_mc_jer = generate_yaml(
    hists, mc_selection,
    "/vols/cms/sdb15/Analysis/ZinvWidth/databases/skims/2019/09_Sep/14_skims/mc_jer/*.h5",
    tables={"jerSF10": "Events_jerSF10", "jerSF20": "Events_jerSF20", "jerSF30": "Events_jerSF30", "jerSF40": "Events_jerSF40", "jerSF50": "Events_jerSF50", "jerSF60": "Events_jerSF60", "jerSF70": "Events_jerSF70", "jerSF80": "Events_jerSF80", "jerSF90": "Events_jerSF90"},
    mc=True,
)
with open("configs/mc_jer.yaml", "w") as f:
    yaml.dump(cfg_mc_jer, f, indent=4)
    
cfg_mc_unclust = generate_yaml(
    hists, mc_selection,
    "/vols/cms/sdb15/Analysis/ZinvWidth/databases/skims/2019/09_Sep/14_skims/mc_unclust/*.h5",
    tables={"unclust10": "Events_unclust10", "unclust20": "Events_unclust20", "unclust30": "Events_unclust30", "unclust40": "Events_unclust40", "unclust50": "Events_unclust50", "unclust60": "Events_unclust60", "unclust70": "Events_unclust70", "unclust80": "Events_unclust80", "unclust90": "Events_unclust90"},
    mc=True,
)
with open("configs/mc_unclust.yaml", "w") as f:
    yaml.dump(cfg_mc_unclust, f, indent=4)
    
cfg_mc_lepscales = generate_yaml(
    hists, mc_selection,
    "/vols/cms/sdb15/Analysis/ZinvWidth/databases/skims/2019/09_Sep/14_skims/mc_lepscales/*.h5",
    tables={"eleEnergyScaleUp": "Events_eleEnergyScaleup", "eleEnergyScaleDown": "Events_eleEnergyScaledown", "muonPtScaleUp": "Events_muonPtScaleup", "muonPtScaleDown": "Events_muonPtScaledown", "photonEnergyScaleUp": "Events_photonEnergyScaleup", "photonEnergyScaleDown": "Events_photonEnergyScaledown", "tauPtScaleUp": "Events_tauPtScaleup", "tauPtScaleDown": "Events_tauPtScaledown"},
    mc=True,
)
with open("configs/mc_lepscales.yaml", "w") as f:
    yaml.dump(cfg_mc_lepscales, f, indent=4)

In [7]:
!~/Scripts/batch/QSTAT.py


 queue   |   free |   used |   total
---------+--------+--------+---------
 hep.q   |    440 |      0 |     440
 gpu.q   |      7 |      0 |       7
 fw.q    |      4 |      0 |       4

No jobs


Best to run the following in the terminal just in case you close your browser or it crashes. Hence the commenting

In [8]:
#zdb.modules.multi_analyse(
#    ["configs/data.yaml", "configs/mc.yaml", "configs/mc_jes.yaml", "configs/mc_jer.yaml", "configs/mc_unclust.yaml", "configs/mc_lepscales.yaml"],
#    outputs=[
#        "hists_wjets_pred_v3.h5:DataAggEvents", "hists_wjets_pred_v3.h5:MCAggEvents",
#        "hists_wjets_pred_v3.h5:MCAggEvents_jes", "hists_wjets_pred_v3.h5:MCAggEvents_jer",
#        "hists_wjets_pred_v3.h5:MCAggEvents_unclust", "hists_wjets_pred_v3.h5:MCAggEvents_lepscales",
#    ],
#    mode='sge',
#    ncores=-1,
#    batch_opts="-q hep.q -l h_rt=3:0:0 -l h_vmem=12G",
#    chunksize=400_000,
#    merge_opts={"mode": "sge", "ncores": 10, "batch_opts": "-q hep.q"},
#)